<a href="https://colab.research.google.com/github/O-Kpy/Project/blob/main/%EB%89%B4%EC%8A%A4%EA%B8%B0%EC%82%AC_%ED%81%AC%EB%A1%A4%EB%9F%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import re

'''''''''''''''''''''''''''''''''''''''''''''''''''''''''
< naver 뉴스 검색시 리스트 크롤링하는 프로그램 > _select사용
- 크롤링 해오는 것 : 링크,제목,신문사,내용요약본
- 내용요약본  -> 정제 작업 필요
- 리스트 -> 딕셔너리 -> df -> 엑셀로 저장
'''''''''''''''''''''

# 각 크롤링 결과 저장하기 위한 리스트 선언
title_text=[]
link_text=[]
source_text=[]
contents_text=[]
result={}

RESULT_PATH ='/content/'  # 결과 저장할 경로

# 내용 전처리
def contents_cleansing(contents):
    first_cleansing_contents = re.sub('<dl>.*?</a> </div> </dd> <dd>', '',str(contents)).strip()  # 앞에 필요없는 부분 제거
    second_cleansing_contents = re.sub('<ul class="relation_lst">.*?</dd>', '', first_cleansing_contents).strip()  # 뒤에 필요없는 부분 제거 (새끼 기사)

 
    third_cleansing_contents = re.sub('<.+?>', '', second_cleansing_contents).strip()
    contents_text.append(third_cleansing_contents)
    #print(contents_text)

# 크롤링 시작
def crawler(maxpage,query,sort,s_date,e_date):
    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    page = 1
    maxpage_t =(int(maxpage)-1)*10+1
    while page <= maxpage_t:
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        response = requests.get(url)
        html = response.text

        # BeautifulSoup 인자값 지정
        soup = BeautifulSoup(html, 'html.parser')

        # <a>태그에서 제목과 링크주소 (a 태그 중 class 명이 news_tit인 것)
        atags = soup.find_all('a', 'news_tit')
        for atag in atags:
            title = atag.get('title')
            title_text.append(title)     # 제목
            link_text.append(atag['href'])   # 링크주소

        # 신문사 추출 (a 태그 중 class 명이 info press인 것)
        source_lists = soup.find_all('a', 'info press')
        for source_list in source_lists:
            source_text.append(source_list.text)    # 신문사

        # 본문요약본 (a 태그 중 class 명이 api_txt_lines dsc_txt_wrap인 것)
        contents_lists = soup.find_all('a','api_txt_lines dsc_txt_wrap')
        for contents_list in contents_lists:
            contents_cleansing(contents_list) # 본문요약 정제화

        # 모든 리스트 딕셔너리형태로 저장
        result= {"title":title_text ,  "source" : source_text ,"contents": contents_text ,"link":link_text, "subtitle":query }
        df = pd.DataFrame(result)  # df로 변환
        page += 10

    # 새로 만들 파일이름 지정
    outputFileName = f'{query} merging.csv'
    df.to_csv(RESULT_PATH+outputFileName)

# 메인함수
def main():
    info_main = input("="*50+"\n"+"입력 형식에 맞게 입력해주세요."+"\n"+" 시작하시려면 Enter를 눌러주세요."+"\n"+"="*50)
    maxpage = input("최대 크롤링할 페이지 수 입력하시오: ")
    query = input("검색어 입력: ")
    sort = input("뉴스 검색 방식 입력(관련도순=0  최신순=1  오래된순=2): ")
    s_date = input("시작날짜 입력(2019.01.04):")
    e_date = input("끝 날짜 입력(2019.01.05):")
    crawler(maxpage,query,sort,s_date,e_date)

main()


입력 형식에 맞게 입력해주세요.
 시작하시려면 Enter를 눌러주세요.
최대 크롤링할 페이지 수 입력하시오: 2
검색어 입력: 녹색기업
뉴스 검색 방식 입력(관련도순=0  최신순=1  오래된순=2): 0
시작날짜 입력(2019.01.04):2019.01.01
끝 날짜 입력(2019.01.05):2021.05.05


In [47]:
import pandas as pd
cr = pd.read_csv('/content/녹색기업 merging.csv')
cr

,Unnamed: 0,title,source,contents,link,subtitle
0,0,탄소중립도 진영논리? 4대강 파헤친 ‘녹색성장’ 못 버리는 국민의힘,한겨레언론사 선정,200여개 시민사회단체가 연대한 기후위기비상행동의 황인철 집행위원장은 “문재인 정부...,http://www.hani.co.kr/arti/society/environment...,녹색기업
1,1,"쌍용C&E, 시멘트업계 최초로 녹색채권 발행 추진",한국경제,쌍용C&amp;E가 한국신용평가와 한국기업평가로부터 녹색채권 발행을 위한 최고 인증...,https://www.hankyung.com/economy/article/20210...,녹색기업
2,2,무늬만 ‘친환경’ 걸러낸다...ESG 공시·녹색 금융의 기준 ‘택소노미’,한경비즈니스언론사 선정,"한 금융 전문가는 ""현재의 투자 기준 등을 보면 자동차의 핸들을 만들어 납품하는 중...",https://magazine.hankyung.com/business/article...,녹색기업
3,3,"김영진 한독 회장 ""ESG 하루아침에 완성 안돼…기업문화로 정착돼야 성공""",매일경제,착한 기업이 성공하는 좋은 사례 올해 새로 구성한 'ESG 커미티' 7가지 세부분야...,http://news.mk.co.kr/newsRead.php?no=434041&ye...,녹색기업
4,4,'착한 기업'이 인정받는 시대…재계로 확산되는 ESG 경영,아시아경제,기업들은 사내에 ESG 전담조직을 설치하는가 하면 친환경·사회적 사업을 위한 투자 ...,https://view.asiae.co.kr/article/2021050511552...,녹색기업
5,5,"녹색마을 협동조합, 울산 동구청에 성금전달",울산제일일보,사회적기업 녹색마을 협동조합 임용걸 대표는 지난 4일 동구청을 방문해 지역 내 어려...,http://www.ujeil.com/news/articleView.html?idx...,녹색기업
6,6,"""실적보다는 ESG가 기업의 신용등급 가른다""",한국경제,"녹색채권, 사회적 채권, 지속가능채권으로 구분된다. 김 본부장은 “국제사회가 각국 ...",https://www.hankyung.com/finance/article/20210...,녹색기업
7,7,"우리은행, 친환경 우수기업에 '녹색금융' 지원 강화",SBS Biz,"우리은행이 오늘(30일) ESG(환경, 사회, 지배구조) 경영 강화를 위해 한국환경...",https://biz.sbs.co.kr/article_hub/20000013766?...,녹색기업
8,8,"쌍용C&E, 시멘트업계 최초 녹색채권 최고 등급",중소기업신문,쌍용C&amp;E는 한국신용평가와 한국기업평가에서 녹색채권 발행을 위한 최고 인증등...,http://www.smedaily.co.kr/news/articleView.htm...,녹색기업
9,9,"[더벨]지역난방공사, 녹색채권 발행 채비…한기평 인증",더벨,한국지역난방공사는 한국기업평가에서 녹색채권을 대상으로 인증평가를 받고 있다. 한국지...,http://www.thebell.co.kr/front/free/contents/n...,녹색기업
